In [1]:
import numpy as np
import pandas as pd
import re
from matplotlib import pyplot as plt

In [30]:
twitter_data_file_path = "../Dados/Ukr_war_tweets.csv"

df_twitter = pd.read_csv(twitter_data_file_path, dtype = 'unicode')

display(df_twitter)

,Datetime,Tweet Id,Text,Username,Permalink,User,Outlinks,CountLinks,ReplyCount,RetweetCount,LikeCount,QuoteCount,ConversationId,Language,Source,Media,QuotedTweet,MentionedUsers,hashtag,hastag_counts
0,2022-02-24 03:12:47+00:00,1496684505247141897,🇺🇦 Massive explosions rocks #Kharkiv. #Russia ...,IdeologyWars,https://twitter.com/IdeologyWars/status/149668...,https://twitter.com/IdeologyWars,NaN,NaN,2,17,47,1,1496493071495987201,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",[Photo(previewUrl='https://pbs.twimg.com/media...,NaN,NaN,"['#Kharkiv.', '#Russia', '#Ukraine', '#Ukraine...",9
1,2022-02-24 03:18:54+00:00,1496686044275695616,🇺🇦 Kharkiv is officially being struck by major...,IdeologyWars,https://twitter.com/IdeologyWars/status/149668...,https://twitter.com/IdeologyWars,NaN,NaN,2,104,188,8,1496493071495987201,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",[Video(thumbnailUrl='https://pbs.twimg.com/ext...,NaN,NaN,"['#Russia', '#Ukraine', '#UkraineWar', '#Russi...",8
2,2022-02-24 03:22:42+00:00,1496687000375726080,🇺🇦 More angles on that strike in Kharkiv. #Rus...,IdeologyWars,https://twitter.com/IdeologyWars/status/149668...,https://twitter.com/IdeologyWars,NaN,NaN,1,41,75,5,1496493071495987201,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",[Video(thumbnailUrl='https://pbs.twimg.com/ext...,NaN,NaN,"['#Russia', '#Ukraine', '#UkraineWar', '#Russi...",8
3,2022-02-24 03:25:36+00:00,1496687731434565636,🇺🇦 BM-21 Grad strikes opening on #Mariupol cit...,IdeologyWars,https://twitter.com/IdeologyWars/status/149668...,https://twitter.com/IdeologyWars,NaN,NaN,21,407,1099,149,1496493071495987201,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",[Video(thumbnailUrl='https://pbs.twimg.com/ext...,NaN,NaN,"['#Mariupol', '#Russia', '#Ukraine', '#Ukraine...",9
4,2022-02-24 03:27:28+00:00,1496688201242759177,🇺🇦 Damage caused by strike in Kharkiv... #Russ...,IdeologyWars,https://twitter.com/IdeologyWars/status/149668...,https://twitter.com/IdeologyWars,NaN,NaN,7,152,207,22,1496493071495987201,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",[Video(thumbnailUrl='https://pbs.twimg.com/ext...,NaN,NaN,"['#Russia', '#Ukraine', '#UkraineWar', '#Russi...",8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399851,2022-03-27 23:59:26+00:00,1508232258037112834,Russia’s mass media and telecommunications wat...,RenKeNews,https://twitter.com/RenKeNews/status/150823225...,https://twitter.com/RenKeNews,['https://tass.com/world/1428013'],['https://t.co/6Ukollhqfj'],1,0,6,0,1508232258037112834,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,NaN,"['#RussiaUkraineWar', '#Russia', '#Ukraine']",3
399852,2022-03-27 23:59:27+00:00,1508232263930105861,@tony07653102 @CNEWS Pour comprendre un confli...,philmarso,https://twitter.com/philmarso/status/150823226...,https://twitter.com/philmarso,NaN,NaN,0,0,1,0,1508135128706560000,fr,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,"[User(username='CNEWS', id=18396319, displayna...",['#Ukraine'],1
399853,2022-03-27 23:59:34+00:00,1508232290413219841,"Putin ukrajinske vojnike naziva neonacizmom, a...",dicke5,https://twitter.com/dicke5/status/150823229041...,https://twitter.com/dicke5,NaN,NaN,0,0,0,0,1508232290413219841,und,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,NaN,"['#Putin,', '#Ukrajina,', '#Zelenskij']",3
399854,2022-03-27 23:59:44+00:00,1508232332289126401,"Watch: amid war ruins in #Ukraine, unbowed cla...",vsdawson_MUFC,https://twitter.com/vsdawson_MUFC/status/15082...,https://twitter.com/vsdawson_MUFC,['https://www.thestar.com.my/lifestyle/culture...,['https://t.co/qPGVgr6li1'],0,0,0,0,1508232332289126401,en,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,NaN,"['#Ukraine,']",1


In [4]:
attrs = ['Datetime', 'Text', 'Language', 'hashtag']

df_twitter_texts = df_twitter[attrs]

display(df_twitter_texts.head())

,Datetime,Text,Language,hashtag
0,2022-02-24 03:12:47+00:00,🇺🇦 Massive explosions rocks #Kharkiv. #Russia ...,en,"['#Kharkiv.', '#Russia', '#Ukraine', '#Ukraine..."
1,2022-02-24 03:18:54+00:00,🇺🇦 Kharkiv is officially being struck by major...,en,"['#Russia', '#Ukraine', '#UkraineWar', '#Russi..."
2,2022-02-24 03:22:42+00:00,🇺🇦 More angles on that strike in Kharkiv. #Rus...,en,"['#Russia', '#Ukraine', '#UkraineWar', '#Russi..."
3,2022-02-24 03:25:36+00:00,🇺🇦 BM-21 Grad strikes opening on #Mariupol cit...,en,"['#Mariupol', '#Russia', '#Ukraine', '#Ukraine..."
4,2022-02-24 03:27:28+00:00,🇺🇦 Damage caused by strike in Kharkiv... #Russ...,en,"['#Russia', '#Ukraine', '#UkraineWar', '#Russi..."


In [7]:
print(help(re.findall))

Help on function findall in module re:

findall(pattern, string, flags=0)
    Return a list of all non-overlapping matches in the string.
    
    If one or more capturing groups are present in the pattern, return
    a list of groups; this will be a list of tuples if the pattern
    has more than one group.
    
    Empty matches are included in the result.

None


In [38]:
def get_words_list(string: str) -> list:
    pattern = r"\b[A-Za-z#]+"
    
    return re.findall(pattern, string.title()) 

def count_word_frequency(df: pd.DataFrame) -> dict:
    word_freq = dict()
    
    for idx, row in df.iterrows():
        tweet_words = get_words_list(str(row['Text']))
        
        for word in tweet_words:
            if word in word_freq.keys():
                word_freq[word] += 1
            else:
                word_freq[word] = 1
    
    return word_freq

def get_min_freq(word_frequency: dict, min_val: int) -> dict:
    words_list = list(filter(lambda x: word_frequency[x] >= min_val, word_frequency.keys()))

    return {key: word_freq[key] for key in words_list}

In [32]:
a = "Oi eu fui no parque weee AAAA weee eu oi oi Oi oi Oi"

print(x := get_words_list(a))

word_freq = dict()
        
for word in x:
    if word in word_freq.keys():
        word_freq[word] += 1
    else:
        word_freq[word] = 1
        
print(word_freq)

['Oi', 'Eu', 'Fui', 'No', 'Parque', 'Weee', 'Aaaa', 'Weee', 'Eu', 'Oi', 'Oi', 'Oi', 'Oi', 'Oi']
{'Oi': 6, 'Eu': 2, 'Fui': 1, 'No': 1, 'Parque': 1, 'Weee': 2, 'Aaaa': 1}


In [33]:
teste = count_word_frequency(df_twitter_texts.head(15))

print(teste, sep = '\n')

{'Massive': 1, 'Explosions': 1, 'Rocks': 1, 'Kharkiv': 4, 'Russia': 14, 'Ukraine': 13, 'Ukrainewar': 12, 'Russiawar': 15, 'Europe': 11, 'Eu': 11, 'Nato': 11, 'Us': 12, 'Https': 10, 'T': 11, 'Co': 10, 'Tpteuysufo': 1, 'Is': 6, 'Officially': 1, 'Being': 1, 'Struck': 1, 'By': 2, 'Major': 1, 'Strikes': 2, 'Ri': 1, 'More': 1, 'Angles': 1, 'On': 3, 'That': 1, 'Strike': 2, 'In': 5, 'J': 1, 'Bm': 1, 'Grad': 1, 'Opening': 1, 'Mariupol': 2, 'City': 1, 'Bigosz': 1, 'Damage': 1, 'Caused': 1, 'Xsvb': 1, 'Boryspil': 2, 'Airport': 2, 'Kyiv': 3, 'Under': 2, 'Attack': 2, 'Statement': 1, 'From': 1, 'President': 2, 'Biden': 1, 'Vihjro': 1, 'Air': 2, 'Raid': 1, 'Sirens': 1, 'Are': 2, 'Apparently': 1, 'Right': 2, 'Now': 2, 'Wyu': 1, 'Saratov': 1, 'Turboprops': 1, 'Can': 1, 'Be': 1, 'Heard': 1, 'Tu': 1, 'Likely': 1, 'Present': 1, 'Nrvw': 1, 'Ahahahaha': 1, 'Potus': 2, 'Wont': 1, 'Speak': 1, 'Until': 1, 'Tomorrow': 1, 'Don': 1, 'Interrupt': 1, 'His': 1, 'Sleep': 1, 'Let': 1, 'The': 2, 'World': 1, 'Burn': 1, 

In [34]:
word_freq = count_word_frequency(df_twitter_texts)

print(len(word_freq))

95883


In [43]:
word_freq_least100 = get_min_freq(word_freq, 1000)

print(len(word_freq_least100))

114
